In [1]:
# Copyright © 2019-2021 Intel Corporation.
# 
# This software and the related documents are Intel copyrighted
# materials, and your use of them is governed by the express 
# license under which they were provided to you (License). Unless
# the License provides otherwise, you may not use, modify, copy, 
# publish, distribute, disclose or transmit  this software or the
# related documents without Intel's prior written permission.
# 
# This software and the related documents are provided as is, with
# no express or implied warranties, other than those that are 
# expressly stated in the License.

# Path Planning

This tutorial shows how to use the path planning model. Path planning model is based on the planning algorithm modeled after the operational principles of hippocampal place cells. The algorithm infers associations between neurons in a network from the asymmetric effects of STDP on a propagating sequence of spikes.

Consider a path planning problem where an autonomous agent is given a map of an environment and is required to plan paths to destination locations within it. The map is encoded in the spiking network of where neurons represent map locations and synaptic connections represent spatial associations between them. The path between the current location of the agent and one of the specified destinations can be found by propagating a spike wave from the destination and issuing motion commands in the direction of the local synaptic vector. Thus to navigate to the destination after a spike wave propagates, the agent localizes itself with sensory cues, reads the path encoded in the local synaptic vector and
generates successive motion commands towards the destination.

The synaptic weight between any two neurons in the network undergo activity-dependent plasticity based on the following learning rule:

   ![title](figures/path_planning_lr.png)


The spiking thresholds of the neurons are chosen such that a postsynaptic spike is generated in response to multiple presynaptic spikes received within a short window of time. When every neuron is configured with an absolute after-
spike refractory period, a localized cluster of spikes within the grid will generate a sequence of spikes as a result of the distance-dependent synaptic configuration. This is visualized as a two-dimensional propagating wavefront as shown in figure below.
   
   ![title](figures/path_planning_wavefront.png)



In [2]:
import os
import matplotlib.pyplot as plt
from IPython.display import IFrame
from nxsdk_modules.path_planning.src.nxcore.path_planning import PathPlanning
from nxsdk_modules.path_planning.src.visualization.export_graph_wavefront_to_html import ExportGraphWavefrontToHtml
from nxsdk.graph.monitor.probes import SpikeProbeCondition

## 1. Loading the graph

Path planning model needs graph in form of Spiking Neural Network with Neurons representing nodes in the graph and synaptic connection edges. The graph we will use for tutorial is 2d 4x4 graph consisting of 16 neurons with all to all connectivity. After obtaining the graph directory, instantiate the Path Planning model.

In [3]:
graphDir = os.path.abspath('') + "/../data/2d4"
graphDir = os.path.abspath('') + "/../data/2d10"
graph = PathPlanning(graphDir)

## 2. Setting up the network

In this phase the network is setup : graph is read from the given directory and all the required nodesets are configured.

In [4]:
chipInfo, board = graph.setupNetwork()

INFO:PATH_PLANNING:  Number of chips : 1
INFO:PATH_PLANNING:  Reading File : /home/wildandr/projects/nxsdk/nxsdk_modules/path_planning/tutorials/../data/2d10/chip0.binary chipId : 0
INFO:PATH_PLANNING:  numCores: 1


## 3. Setting the destination

Each graph has a destination and source written in the file. In this step we get the source and destination and then set the bias for the destination so that it spikes in the next timestep.

In [5]:
# Read targets
targetList = graph.readTargets(graphDir, 0)
graph.logger.info(
    "Target :"
    " chipId: {}"
    " coreId: {}"
    " compartmentId: {}".format(
        targetList[1]['chipId'],
        targetList[1]['coreId'],
        targetList[1]['compartmentId']))
graph.setTargetBias(targetList[1], 1 << 6 + 1)

INFO:PATH_PLANNING:  Target : chipId: 0 coreId: 0 compartmentId: 99
INFO:PATH_PLANNING:  Setting target Bias: 128 chipId: 0 coreId: 0 compartmentId: 99


## 4. Printing out the obtained source

In [6]:
sourceChipId = targetList[0]['chipId']
sourceCoreId = targetList[0]['coreId']
sourceCompartmentId = targetList[0]['compartmentId']
sourceCore = board.n2Chips[sourceChipId].n2Cores[sourceCoreId]
graph.logger.info(
    "Source :"
    " chipId: {}"
    " coreId: {}"
    " compartmentId: {}".format(
            sourceChipId,
            sourceCoreId,
            sourceCompartmentId))

INFO:PATH_PLANNING:  Source : chipId: 0 coreId: 0 compartmentId: 0


## 5. Setting up the snip

In this step we setup the snip, which creates a snip in the mgmt phase and also creates a channel, which will be used to communicate when the spike from the destination reached the source.

In [7]:
spikeTimeChannel = graph.setupSnip(sourceChipId)

# Define probes
mon = board.monitor

# tStart = 10000000 (any large value so that the probe infrastrucutre on host doesn't kick-in for mgmt read)
# What creating a probe will do, is to create an output axon from cx to lmt counter 0x20
# Probe counters are assigned incrementally.
# If we would have created one more probe its lmt address would have been
# 0x21
customSpikeProbeCond = SpikeProbeCondition(tStart=10000000)
spikeProbesCust = mon.probe(
    sourceCore.cxState,
    [sourceCompartmentId],
    'spike',
    probeCondition=customSpikeProbeCond)

## 6. Running the network

In this step, we first run the network for 1 timestep to allow the destination to spike and then reset the bias of the target, so that it doesn't spikes again. After that we run for arbitary large number of timesteps till the source spikes.


In [8]:
# Run the network
board.run(1)
# Reset target bias to prevent spiking
graph.setTargetBias(targetList[1], 1)
# Continue running the network
board.run(2000, aSync=True)

INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.81:36353
INFO:DRV:      Host server up..............Done 3.60s
INFO:DRV:      Encoding axons/synapses.....Done 0.03s
INFO:DRV:      Compiling Embedded snips....Done 0.64s
INFO:DRV:      Encoding probes.............Done 0.63ms
INFO:DRV:      Booting up..................Done 2.59s
INFO:DRV:      Configuring registers.......Done 0.01s
INFO:DRV:      Transferring probes.........Done 2.95ms
INFO:DRV:      Transferring spikes.........Done 0.56ms
INFO:DRV:      Executing...................Done 1.73ms
INFO:DRV:      Processing timeseries.......Done 1.64ms
INFO:DRV:  Executor: 1 timesteps...........Done 6.91s
INFO:PATH_PLANNING:  Setting target Bias: 1 chipId: 0 coreId: 0 compartmentId: 99
INFO:DRV:      Configuring registers.......Done 4.71ms
INFO:DRV:      Transferring probes.........Done 0.89ms
INFO:DRV:      Transferring spikes.........Done 0.43ms


## 7. Getting the time when the source spikes

We use the channel we created in the snip to obtain the time at which the source spikes. We pause the execution after we know learning epoch has passed after source spiked.

In [9]:
spikeTime = []
# Read spiketime when shortest path is found
spikeTime.append(spikeTimeChannel.read(1))
# Stop excution after epoch is passed
spikeTime.append(spikeTimeChannel.read(1))
board.pause()

## 8. Calculating the Shortest Path

The length of the shortest path is the timestep at which source spikes. 

In [10]:
graph.logger.info("Length of shortest path: {}".format(spikeTime[0][0]))
graph.logger.info("Current timestep: {}".format(spikeTime[1][0]))    

INFO:PATH_PLANNING:  Length of shortest path: 19
INFO:PATH_PLANNING:  Current timestep: 24


## 9. Tracing the Synaptic entries to get the shortest path

We trace the synaptic entries from the source to destination to determine the shortest path.

In [11]:
shortestPath = graph.tracePathThroughSynapseWeights(targetList[0], targetList[1])

INFO:PATH_PLANNING:  List of nodes in the shortest path are:
INFO:PATH_PLANNING:  Node: 0 Chip: 0 Core: 0 Compartment: 0
INFO:PATH_PLANNING:  Node: 1 Chip: 0 Core: 0 Compartment: 10
INFO:PATH_PLANNING:  Node: 2 Chip: 0 Core: 0 Compartment: 11
INFO:PATH_PLANNING:  Node: 3 Chip: 0 Core: 0 Compartment: 21
INFO:PATH_PLANNING:  Node: 4 Chip: 0 Core: 0 Compartment: 22
INFO:PATH_PLANNING:  Node: 5 Chip: 0 Core: 0 Compartment: 23
INFO:PATH_PLANNING:  Node: 6 Chip: 0 Core: 0 Compartment: 24
INFO:PATH_PLANNING:  Node: 7 Chip: 0 Core: 0 Compartment: 25
INFO:PATH_PLANNING:  Node: 8 Chip: 0 Core: 0 Compartment: 26
INFO:PATH_PLANNING:  Node: 9 Chip: 0 Core: 0 Compartment: 27
INFO:PATH_PLANNING:  Node: 10 Chip: 0 Core: 0 Compartment: 28
INFO:PATH_PLANNING:  Node: 11 Chip: 0 Core: 0 Compartment: 29
INFO:PATH_PLANNING:  Node: 12 Chip: 0 Core: 0 Compartment: 39
INFO:PATH_PLANNING:  Node: 13 Chip: 0 Core: 0 Compartment: 49
INFO:PATH_PLANNING:  Node: 14 Chip: 0 Core: 0 Compartment: 59
INFO:PATH_PLANNING: 

In [12]:
ExportGraphWavefrontToHtml(graph).publish()
IFrame(src="wavefront.html", height='500px', width='100%')

In [13]:
board.disconnect()